# Analytic Geometry (Lecture 3)

**Course:** Mathematics for Machine Learning  
**Instructor:** Mohammed Alnemari

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Compute **norms** (L1, L2, L-infinity) and visualize unit balls
2. Work with **inner products** and verify positive definiteness
3. Calculate **distances** and **angles** between vectors
4. Check **orthogonality** and build orthogonal complements
5. Perform **projections** onto lines and subspaces
6. Implement the **Gram-Schmidt** orthogonalization process
7. Construct and apply **rotation** matrices

---

## Google Colab Ready!

This notebook works perfectly in Google Colab. All required libraries are pre-installed!

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

# For nicer plots
plt.rcParams['figure.figsize'] = (6, 5)
plt.rcParams['figure.dpi'] = 100

print("NumPy version:", np.__version__)
print("Libraries imported successfully!")

---

# Part 1: Norms

A **norm** assigns a non-negative length or size to each vector in a vector space.

Common norms for a vector $\mathbf{x} = (x_1, x_2, \ldots, x_n)$:

- **L1 norm (Manhattan):** $\|\mathbf{x}\|_1 = \sum_{i=1}^{n} |x_i|$
- **L2 norm (Euclidean):** $\|\mathbf{x}\|_2 = \sqrt{\sum_{i=1}^{n} x_i^2}$
- **L-infinity norm (Max):** $\|\mathbf{x}\|_\infty = \max_i |x_i|$

## 1.1 Computing Norms with NumPy

In [ ]:
# Define a vector
x = np.array([3, -4, 1, 2])
print("Vector x =", x)
print()

# L1 norm (Manhattan norm)
l1_norm = np.linalg.norm(x, ord=1)
print("L1 norm  ||x||_1   =", l1_norm)

# L2 norm (Euclidean norm)
l2_norm = np.linalg.norm(x, ord=2)
print("L2 norm  ||x||_2   =", l2_norm)

# L-infinity norm (Max norm)
linf_norm = np.linalg.norm(x, ord=np.inf)
print("Linf norm ||x||_inf =", linf_norm)
print()

# Verify by hand
print("Manual L1:  ", np.sum(np.abs(x)))
print("Manual L2:  ", np.sqrt(np.sum(x**2)))
print("Manual Linf:", np.max(np.abs(x)))

## 1.2 Visualizing Unit Balls in 2D

The **unit ball** for a norm is the set of all points with norm $\leq 1$:
$\{ \mathbf{x} : \|\mathbf{x}\| \leq 1 \}$

In [ ]:
# Generate a grid of points in 2D
grid = np.linspace(-1.5, 1.5, 500)
X, Y = np.meshgrid(grid, grid)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# L1 unit ball: |x| + |y| <= 1 (diamond)
Z1 = np.abs(X) + np.abs(Y)
axes[0].contourf(X, Y, Z1, levels=[0, 1], colors=['#3498db'], alpha=0.5)
axes[0].contour(X, Y, Z1, levels=[1], colors=['#2c3e50'], linewidths=2)
axes[0].set_title('L1 Unit Ball (Diamond)', fontsize=13)
axes[0].set_aspect('equal')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(0, color='k', linewidth=0.5)
axes[0].axvline(0, color='k', linewidth=0.5)

# L2 unit ball: sqrt(x^2 + y^2) <= 1 (circle)
Z2 = np.sqrt(X**2 + Y**2)
axes[1].contourf(X, Y, Z2, levels=[0, 1], colors=['#2ecc71'], alpha=0.5)
axes[1].contour(X, Y, Z2, levels=[1], colors=['#2c3e50'], linewidths=2)
axes[1].set_title('L2 Unit Ball (Circle)', fontsize=13)
axes[1].set_aspect('equal')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(0, color='k', linewidth=0.5)
axes[1].axvline(0, color='k', linewidth=0.5)

# L-infinity unit ball: max(|x|, |y|) <= 1 (square)
Zinf = np.maximum(np.abs(X), np.abs(Y))
axes[2].contourf(X, Y, Zinf, levels=[0, 1], colors=['#e74c3c'], alpha=0.5)
axes[2].contour(X, Y, Zinf, levels=[1], colors=['#2c3e50'], linewidths=2)
axes[2].set_title('L-inf Unit Ball (Square)', fontsize=13)
axes[2].set_aspect('equal')
axes[2].grid(True, alpha=0.3)
axes[2].axhline(0, color='k', linewidth=0.5)
axes[2].axvline(0, color='k', linewidth=0.5)

plt.suptitle('Unit Balls for Different Norms', fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---

# Part 2: Inner Products

An **inner product** $\langle \mathbf{x}, \mathbf{y} \rangle$ is a generalization of the dot product.

- **Standard dot product:** $\langle \mathbf{x}, \mathbf{y} \rangle = \mathbf{x}^\top \mathbf{y} = \sum_i x_i y_i$
- **Generalized inner product:** $\langle \mathbf{x}, \mathbf{y} \rangle_A = \mathbf{x}^\top A \mathbf{y}$ where $A$ is symmetric positive definite

## 2.1 Dot Product with np.dot

In [ ]:
# Define two vectors
x = np.array([1, 2, 3])
y = np.array([4, 5, 6])

# Compute dot product (three equivalent ways)
dot1 = np.dot(x, y)
dot2 = x @ y
dot3 = np.sum(x * y)

print("x =", x)
print("y =", y)
print()
print("np.dot(x, y) =", dot1)
print("x @ y        =", dot2)
print("sum(x * y)   =", dot3)
print()
print("All methods agree:", dot1 == dot2 == dot3)

## 2.2 Custom Inner Product with Matrix A

Given a symmetric positive definite matrix $A$, we can define an inner product:

$$\langle \mathbf{x}, \mathbf{y} \rangle_A = \mathbf{x}^\top A \, \mathbf{y}$$

In [ ]:
# Define a symmetric positive definite matrix A
A = np.array([[2, 1],
              [1, 3]])

# Define two 2D vectors
x = np.array([1, 0])
y = np.array([0, 1])

# Standard dot product
std_dot = np.dot(x, y)
print("Standard dot product: <x, y> =", std_dot)

# Custom inner product: x^T A y
custom_ip = x @ A @ y
print("Custom inner product: <x, y>_A =", custom_ip)
print()

# The custom inner product can detect 'similarity' that
# the standard dot product misses
print("Note: Standard dot product says x and y are orthogonal.")
print("The A-inner product shows they are NOT A-orthogonal.")

## 2.3 Checking Positive Definiteness

A symmetric matrix $A$ is **positive definite** if all its eigenvalues are positive.
This is required for $A$ to define a valid inner product.

In [ ]:
# Check if a matrix is positive definite
def is_positive_definite(M):
    """Check if matrix M is symmetric positive definite."""
    # Check symmetry
    if not np.allclose(M, M.T):
        return False, "Not symmetric"
    # Check eigenvalues
    eigenvalues = np.linalg.eigvalsh(M)
    if np.all(eigenvalues > 0):
        return True, eigenvalues
    else:
        return False, eigenvalues

# Test with a positive definite matrix
A = np.array([[2, 1],
              [1, 3]])
result, info = is_positive_definite(A)
print("Matrix A:")
print(A)
print("Positive definite?", result)
print("Eigenvalues:", info)
print()

# Test with a non-positive-definite matrix
B = np.array([[1, 2],
              [2, 1]])
result, info = is_positive_definite(B)
print("Matrix B:")
print(B)
print("Positive definite?", result)
print("Eigenvalues:", info)

---

# Part 3: Distances and Angles

Using inner products and norms, we can compute distances and angles between vectors.

## 3.1 Euclidean Distance

In [ ]:
# Define two points
p = np.array([1, 2, 3])
q = np.array([4, 6, 3])

# Euclidean distance = ||p - q||_2
dist = np.linalg.norm(p - q)
print("Point p =", p)
print("Point q =", q)
print()
print("Euclidean distance d(p, q) =", dist)
print("Manual calculation:", np.sqrt(np.sum((p - q)**2)))

## 3.2 Cosine Similarity and Angle Between Vectors

The angle $\theta$ between two vectors is given by:

$$\cos \theta = \frac{\langle \mathbf{x}, \mathbf{y} \rangle}{\|\mathbf{x}\| \cdot \|\mathbf{y}\|}$$

In [ ]:
# Define two vectors
x = np.array([1, 0])
y = np.array([1, 1])

# Cosine similarity
cos_sim = np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))
print("x =", x)
print("y =", y)
print()
print("Cosine similarity:", cos_sim)

# Angle in radians and degrees
angle_rad = np.arccos(np.clip(cos_sim, -1, 1))
angle_deg = np.degrees(angle_rad)
print("Angle (radians):", angle_rad)
print("Angle (degrees):", angle_deg)
print()

# Another example
a = np.array([1, 0])
b = np.array([0, 1])
cos_ab = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
angle_ab = np.degrees(np.arccos(np.clip(cos_ab, -1, 1)))
print("Angle between [1,0] and [0,1]:", angle_ab, "degrees (perpendicular)")

## 3.3 Verify the Cauchy-Schwarz Inequality

$$|\langle \mathbf{x}, \mathbf{y} \rangle| \leq \|\mathbf{x}\| \cdot \|\mathbf{y}\|$$

Equality holds if and only if $\mathbf{x}$ and $\mathbf{y}$ are linearly dependent.

In [ ]:
# Test Cauchy-Schwarz with random vectors
np.random.seed(42)

print("Cauchy-Schwarz Inequality: |<x,y>| <= ||x|| * ||y||")
print("=" * 55)

for i in range(5):
    x = np.random.randn(4)
    y = np.random.randn(4)
    
    lhs = np.abs(np.dot(x, y))           # |<x, y>|
    rhs = np.linalg.norm(x) * np.linalg.norm(y)  # ||x|| * ||y||
    
    print(f"Trial {i+1}: |<x,y>| = {lhs:.4f}  <=  ||x||*||y|| = {rhs:.4f}  "
          f"  Holds: {lhs <= rhs + 1e-10}")

print()

# Equality case: x and y are linearly dependent
x = np.array([1, 2, 3])
y = 5 * x  # y is a scalar multiple of x
lhs = np.abs(np.dot(x, y))
rhs = np.linalg.norm(x) * np.linalg.norm(y)
print("Equality case (y = 5x):")
print(f"  |<x,y>| = {lhs:.4f}  ==  ||x||*||y|| = {rhs:.4f}")
print(f"  Equal? {np.isclose(lhs, rhs)}")

---

# Part 4: Orthogonality

Two vectors $\mathbf{x}$ and $\mathbf{y}$ are **orthogonal** if $\langle \mathbf{x}, \mathbf{y} \rangle = 0$.

## 4.1 Checking Orthogonality

In [ ]:
# Define vectors
u = np.array([1, 0, 0])
v = np.array([0, 1, 0])
w = np.array([1, 1, 0])

def check_orthogonal(a, b, name_a="a", name_b="b"):
    """Check if two vectors are orthogonal."""
    dot = np.dot(a, b)
    is_orth = np.isclose(dot, 0)
    print(f"<{name_a}, {name_b}> = {dot:.4f}  ->  Orthogonal? {is_orth}")
    return is_orth

print("u =", u)
print("v =", v)
print("w =", w)
print()

check_orthogonal(u, v, "u", "v")
check_orthogonal(u, w, "u", "w")
check_orthogonal(v, w, "v", "w")

## 4.2 Orthogonal Complement

The **orthogonal complement** of a subspace $U$ is the set of all vectors orthogonal to every vector in $U$.

We can find it using the null space of the matrix whose rows span $U$.

In [ ]:
# Define a subspace spanned by two vectors in R^3
u1 = np.array([1, 0, 1])
u2 = np.array([0, 1, 1])

# Stack them as rows of a matrix
U = np.vstack([u1, u2])
print("Subspace basis (as rows):")
print(U)
print()

# Find orthogonal complement using SVD (null space of U)
# The null space is spanned by right singular vectors
# corresponding to zero singular values
_, S, Vt = np.linalg.svd(U)
print("Singular values:", S)

# Vectors in Vt beyond the rank form the null space
rank = np.sum(S > 1e-10)
null_space = Vt[rank:]  # orthogonal complement basis

print("Rank of U:", rank)
print()
print("Orthogonal complement basis:")
print(null_space)
print()

# Verify orthogonality
for i, nc in enumerate(null_space):
    print(f"<u1, null_vec_{i+1}> = {np.dot(u1, nc):.6f}")
    print(f"<u2, null_vec_{i+1}> = {np.dot(u2, nc):.6f}")

## 4.3 Orthogonal Matrix Verification

A square matrix $Q$ is **orthogonal** if $Q^\top Q = Q Q^\top = I$.

Equivalently, its columns (and rows) form an orthonormal set.

In [ ]:
# Construct an orthogonal matrix using a rotation
theta = np.pi / 4  # 45 degrees
Q = np.array([[np.cos(theta), -np.sin(theta)],
              [np.sin(theta),  np.cos(theta)]])

print("Orthogonal matrix Q (45-degree rotation):")
print(Q)
print()

# Verify Q^T Q = I
QtQ = Q.T @ Q
print("Q^T @ Q:")
print(QtQ)
print()

# Check if close to identity
I = np.eye(2)
print("Q^T Q == I?", np.allclose(QtQ, I))
print()

# Verify determinant is +1 or -1
det_Q = np.linalg.det(Q)
print("det(Q) =", det_Q)
print("det(Q) = +/-1?", np.isclose(np.abs(det_Q), 1))
print()

# Column norms should all be 1
for j in range(Q.shape[1]):
    print(f"||column {j+1}|| = {np.linalg.norm(Q[:, j]):.6f}")

---

# Part 5: Projections

**Projection** maps a vector onto a subspace, finding the closest point in that subspace.

- Projection onto a line (1D subspace spanned by $\mathbf{b}$): $\text{proj}_{\mathbf{b}}(\mathbf{x}) = \frac{\mathbf{b}^\top \mathbf{x}}{\mathbf{b}^\top \mathbf{b}} \mathbf{b}$

- Projection onto a general subspace (columns of $B$): $\text{proj}_B(\mathbf{x}) = B(B^\top B)^{-1} B^\top \mathbf{x}$

## 5.1 Projection onto a Line

In [ ]:
# Projection of x onto the line spanned by b
x = np.array([3, 4])
b = np.array([1, 1])

# proj_b(x) = (b^T x / b^T b) * b
scalar = np.dot(b, x) / np.dot(b, b)
proj = scalar * b

print("Vector x =", x)
print("Line direction b =", b)
print()
print("Projection coefficient (lambda):", scalar)
print("Projection proj_b(x) =", proj)
print()

# The residual should be orthogonal to b
residual = x - proj
print("Residual (x - proj):", residual)
print("<residual, b> =", np.dot(residual, b), "(should be 0)")

## 5.2 Projection onto a General Subspace

$$\pi_B(\mathbf{x}) = B(B^\top B)^{-1} B^\top \mathbf{x}$$

In [ ]:
# Define subspace basis (columns of B) in R^3
B = np.array([[1, 0],
              [0, 1],
              [0, 0]])  # xy-plane in R^3

# Vector to project
x = np.array([2, 3, 5])

# Projection matrix: P = B (B^T B)^{-1} B^T
P = B @ np.linalg.inv(B.T @ B) @ B.T
print("Projection matrix P:")
print(P)
print()

# Compute projection
proj = P @ x
print("Vector x =", x)
print("Projection onto xy-plane:", proj)
print()

# Verify: P is idempotent (P^2 = P)
print("P^2 == P?", np.allclose(P @ P, P))

# Verify: P is symmetric
print("P == P^T?", np.allclose(P, P.T))
print()

# Residual should be orthogonal to the subspace
residual = x - proj
print("Residual:", residual)
print("B^T @ residual =", B.T @ residual, "(should be zero vector)")

## 5.3 Visualize Projection in 2D

In [ ]:
# Visualize projection of x onto line through b
x = np.array([3, 4])
b = np.array([2, 1])

# Compute projection
proj = (np.dot(b, x) / np.dot(b, b)) * b
residual = x - proj

fig, ax = plt.subplots(1, 1, figsize=(7, 7))

# Draw the line through b (extended)
t = np.linspace(-1, 3, 100)
line_x = t * b[0]
line_y = t * b[1]
ax.plot(line_x, line_y, 'k--', alpha=0.3, label='Subspace (line)')

# Draw vectors
ax.annotate('', xy=x, xytext=(0, 0),
            arrowprops=dict(arrowstyle='->', color='blue', lw=2))
ax.annotate('', xy=proj, xytext=(0, 0),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))
ax.annotate('', xy=x, xytext=proj,
            arrowprops=dict(arrowstyle='->', color='red', lw=2, linestyle='dashed'))

# Labels
ax.text(x[0] + 0.1, x[1] + 0.1, '$\mathbf{x}$', fontsize=14, color='blue')
ax.text(proj[0] - 0.3, proj[1] - 0.4, '$\mathrm{proj}(\mathbf{x})$',
        fontsize=12, color='green')
ax.text((x[0] + proj[0]) / 2 + 0.15, (x[1] + proj[1]) / 2,
        'residual', fontsize=11, color='red')

# Draw right-angle marker
corner_size = 0.2
r_unit = residual / np.linalg.norm(residual)
b_unit = b / np.linalg.norm(b)
corner = np.array([proj + corner_size * r_unit,
                   proj + corner_size * r_unit + corner_size * b_unit,
                   proj + corner_size * b_unit])
ax.plot(corner[:, 0], corner[:, 1], 'k-', lw=1)

ax.set_xlim(-0.5, 5)
ax.set_ylim(-0.5, 5)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.axhline(0, color='k', linewidth=0.5)
ax.axvline(0, color='k', linewidth=0.5)
ax.set_title('Projection of x onto a Line', fontsize=14)
ax.legend(loc='upper left')
plt.tight_layout()
plt.show()

---

# Part 6: Gram-Schmidt Orthogonalization

The **Gram-Schmidt process** takes a set of linearly independent vectors and produces an orthonormal set.

Given vectors $\{\mathbf{b}_1, \ldots, \mathbf{b}_n\}$, Gram-Schmidt produces $\{\mathbf{q}_1, \ldots, \mathbf{q}_n\}$ such that:
- $\langle \mathbf{q}_i, \mathbf{q}_j \rangle = 0$ for $i \neq j$
- $\|\mathbf{q}_i\| = 1$ for all $i$

## 6.1 Gram-Schmidt from Scratch

In [ ]:
def gram_schmidt(V):
    """
    Perform Gram-Schmidt orthogonalization.
    
    Parameters:
        V : array of shape (n, k) where columns are input vectors
    
    Returns:
        Q : array of shape (n, k) with orthonormal columns
    """
    n, k = V.shape
    Q = np.zeros((n, k))
    
    for j in range(k):
        # Start with the original vector
        v = V[:, j].copy()
        
        # Subtract projections onto all previous orthonormal vectors
        for i in range(j):
            v = v - np.dot(Q[:, i], V[:, j]) * Q[:, i]
        
        # Normalize
        norm_v = np.linalg.norm(v)
        if norm_v < 1e-10:
            raise ValueError(f"Vector {j+1} is linearly dependent on previous vectors.")
        Q[:, j] = v / norm_v
    
    return Q


# Test with 3 vectors in R^3
V = np.array([[1, 1, 0],
              [1, 0, 1],
              [0, 1, 1]], dtype=float)

print("Original vectors (columns of V):")
print(V)
print()

Q = gram_schmidt(V)
print("Orthonormal vectors (columns of Q):")
print(Q)
print()

# Verify orthonormality: Q^T Q should be identity
print("Q^T @ Q (should be identity):")
print(np.round(Q.T @ Q, 10))

## 6.2 Compare with scipy.linalg.qr

The QR decomposition factors a matrix $A = QR$ where $Q$ is orthogonal and $R$ is upper triangular.
The columns of $Q$ are the result of Gram-Schmidt applied to the columns of $A$.

In [ ]:
# QR decomposition using SciPy
Q_scipy, R_scipy = linalg.qr(V, mode='economic')

print("SciPy Q (may differ in sign):")
print(Q_scipy)
print()
print("Our Gram-Schmidt Q:")
print(Q)
print()

# The columns may differ by a sign, but the subspaces are the same
# Check by verifying Q^T Q_scipy has entries +1 or -1 on diagonal
print("Absolute column dot products (should be 1.0):")
for j in range(Q.shape[1]):
    dot = np.abs(np.dot(Q[:, j], Q_scipy[:, j]))
    print(f"  |<q_{j+1}, q_scipy_{j+1}>| = {dot:.6f}")

print()
print("R matrix from QR decomposition:")
print(np.round(R_scipy, 6))

print()
print("Verify: Q @ R == V?")
print(np.allclose(Q_scipy @ R_scipy, V))

---

# Part 7: Rotations

A **rotation matrix** in 2D rotates vectors by an angle $\theta$ counterclockwise:

$$R(\theta) = \begin{pmatrix} \cos\theta & -\sin\theta \\ \sin\theta & \cos\theta \end{pmatrix}$$

Properties:
- $R$ is orthogonal: $R^\top R = I$
- $\det(R) = 1$ (proper rotation)
- Rotations preserve lengths and angles

## 7.1 2D Rotation Matrix

In [ ]:
def rotation_matrix_2d(theta):
    """Create a 2D rotation matrix for angle theta (in radians)."""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s],
                     [s,  c]])

# Rotate a vector by 90 degrees
theta = np.pi / 2  # 90 degrees
R = rotation_matrix_2d(theta)

v = np.array([1, 0])
v_rotated = R @ v

print(f"Rotation angle: {np.degrees(theta)} degrees")
print("R =")
print(np.round(R, 6))
print()
print("Original vector:  ", v)
print("Rotated vector:   ", np.round(v_rotated, 6))
print()

# Verify properties
print("det(R) =", np.round(np.linalg.det(R), 6))
print("R^T R = I?", np.allclose(R.T @ R, np.eye(2)))
print("||v|| =", np.linalg.norm(v))
print("||Rv|| =", np.linalg.norm(v_rotated))
print("Length preserved?", np.isclose(np.linalg.norm(v), np.linalg.norm(v_rotated)))

## 7.2 Visualize Rotation of Vectors

In [ ]:
# Define several vectors to rotate
vectors = np.array([[2, 0],
                    [1, 2],
                    [0, 1.5],
                    [-1, 1]])

# Rotation angle: 45 degrees
theta = np.pi / 4
R = rotation_matrix_2d(theta)

fig, ax = plt.subplots(1, 1, figsize=(8, 8))

colors = ['#e74c3c', '#3498db', '#2ecc71', '#9b59b6']

for i, v in enumerate(vectors):
    v_rot = R @ v
    
    # Original vector (solid arrow)
    ax.annotate('', xy=v, xytext=(0, 0),
                arrowprops=dict(arrowstyle='->', color=colors[i], lw=2))
    ax.text(v[0] + 0.05, v[1] + 0.1, f'$v_{i+1}$',
            fontsize=12, color=colors[i])
    
    # Rotated vector (dashed arrow)
    ax.annotate('', xy=v_rot, xytext=(0, 0),
                arrowprops=dict(arrowstyle='->', color=colors[i],
                                lw=2, linestyle='dashed'))
    ax.text(v_rot[0] + 0.05, v_rot[1] + 0.1, f"$v_{i+1}'$",
            fontsize=12, color=colors[i], style='italic')

# Draw rotation arc for one vector
arc_angles = np.linspace(0, theta, 30)
arc_r = 0.8
ax.plot(arc_r * np.cos(arc_angles), arc_r * np.sin(arc_angles),
        'k-', lw=1.5)
ax.text(0.85, 0.3, r'$\theta = 45°$', fontsize=11)

ax.set_xlim(-3, 3)
ax.set_ylim(-1, 3.5)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.axhline(0, color='k', linewidth=0.5)
ax.axvline(0, color='k', linewidth=0.5)
ax.set_title(f'2D Rotation by {np.degrees(theta):.0f} Degrees\n'
             f'(solid = original, dashed = rotated)', fontsize=14)
plt.tight_layout()
plt.show()

---

# Summary

In this notebook we covered the key concepts of **Analytic Geometry** for machine learning:

| Topic | Key Idea |
|-------|----------|
| **Norms** | Measure vector length (L1, L2, L-inf) |
| **Inner Products** | Generalized dot product; requires positive definite matrix |
| **Distances & Angles** | Euclidean distance, cosine similarity, Cauchy-Schwarz |
| **Orthogonality** | Vectors with zero inner product; orthogonal matrices |
| **Projections** | Closest point in a subspace; $B(B^\top B)^{-1}B^\top$ |
| **Gram-Schmidt** | Produces orthonormal basis from independent vectors |
| **Rotations** | Length-preserving linear maps; orthogonal with det = 1 |

---

# Practice Exercises

Try these on your own:

---

**Exercise 1: Norms and Unit Vectors**

Given the vector $\mathbf{v} = (3, -1, 2, -4, 5)$:
1. Compute the L1, L2, and L-infinity norms.
2. Normalize $\mathbf{v}$ to have unit L2 norm and verify that $\|\hat{\mathbf{v}}\|_2 = 1$.
3. For which $p$-norm is the norm of $\mathbf{v}$ the largest? The smallest?

---

**Exercise 2: Inner Products and Angles**

Let $\mathbf{a} = (1, 2, 3)$ and $\mathbf{b} = (4, -5, 6)$.
1. Compute the standard dot product $\langle \mathbf{a}, \mathbf{b} \rangle$.
2. Find the angle between $\mathbf{a}$ and $\mathbf{b}$ in degrees.
3. Define the matrix $A = \begin{pmatrix} 2 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 3 \end{pmatrix}$ and compute the generalized inner product $\mathbf{a}^\top A \mathbf{b}$. Is this matrix positive definite?

---

**Exercise 3: Projections**

Consider the vector $\mathbf{x} = (1, 2, 3)$ and the subspace spanned by $\mathbf{b}_1 = (1, 1, 0)$ and $\mathbf{b}_2 = (0, 1, 1)$.
1. Form the matrix $B = [\mathbf{b}_1 \,|\, \mathbf{b}_2]$ and compute the projection matrix $P = B(B^\top B)^{-1}B^\top$.
2. Compute the projection $\hat{\mathbf{x}} = P\mathbf{x}$.
3. Verify that the residual $\mathbf{x} - \hat{\mathbf{x}}$ is orthogonal to both $\mathbf{b}_1$ and $\mathbf{b}_2$.

---

**Exercise 4: Gram-Schmidt and Rotations**

1. Apply the Gram-Schmidt process to the vectors $\mathbf{v}_1 = (1, 1, 0)$, $\mathbf{v}_2 = (1, 0, 1)$, $\mathbf{v}_3 = (0, 1, 1)$ and verify that the result is an orthonormal set.
2. Create a rotation matrix that rotates by $60°$ and apply it to the vector $(1, 0)$. Verify that the resulting vector has the same length as the original.
3. Show that applying two successive rotations $R(\alpha)$ and $R(\beta)$ is the same as $R(\alpha + \beta)$. Test with $\alpha = 30°$ and $\beta = 45°$.

---

**Course:** Mathematics for Machine Learning  
**Instructor:** Mohammed Alnemari